Take the GOterms with unique info, Bonferroni correction etc. Reformat into something with nodes and links, for displaying with D3

In [2]:
import json
with open("./GOtermsCorrectedUnique.json") as flines:
    enrichment = json.load(flines)
    
%load_ext autoreload
%autoreload 2
import proteinnetworks
from bson.objectid import ObjectId
db = proteinnetworks.database.Database()


password: 8moMlmkRnZurXsl


### Data structure format.
Array of [isoClass, GOterms], where isoClass is a list of PDBs, and GOterms is a dict including "GOlabel" and "pCorr"

In [36]:
# For each isoClass, generate the supernetwork for the first pdb (should be the same for all).
data = []
nodes = []
links = []
for group, isoClass in enumerate(enrichment):
    pdb = isoClass['isoClass'][0]
    # Extract/Generate the edgelist
    inputArgs = {
        "scaling": 4.5,
        "edgelisttype": "residue",
        "hydrogenstatus": "noH",
        "pdbref": pdb,
        "database": db
    }
    proteinNetwork = proteinnetworks.network.Network(**inputArgs)
    # Extract/Generate the partition
    partitionArgs = {
        "pdbref": pdb,
        "edgelistid": ObjectId(proteinNetwork.edgelistid),
        "detectionmethod": "Infomap",
        "N": 100,
        "database": db
    }
    partition = proteinnetworks.partition.Partition(**partitionArgs)
    superNetwork = proteinnetworks.insight.SuperNetwork(
        inputpartition=partition)
    
    # Add the data to be tabulated
    isoClassInfo = {"group": group, "PDBs": isoClass['isoClass'], "GOterms": isoClass['GOterms']}
    data.append(isoClassInfo)    
    #Add the nodes and links
    # For the nodes: get the maximum value currently in "nodes"
    # Must offset by this amount
    if nodes:
        offset = max([x["name"] for x in nodes])
    else:
        offset = 0

    # Get the largest node in superNetwork.data
    try:
        largestNode = max(map(max, [[i, j] for i, j, _ in superNetwork.data]))
    except ValueError as e:
        #FOR NOW, THIS IS BAD!
        continue
    isoClassNodes = [{"name": i + 1 + offset, "group": group} for i in range(largestNode)]
    
    isoClassLinks = [{"source": i - 1 + offset,
                      "target": j - 1 + offset,
                      "value": 1} for i, j, _ in superNetwork.data]
    
    nodes += isoClassNodes
    links += isoClassLinks
    
d3json = {"nodes": nodes, "links": links, "data": data}

with open("GOtermsD3.json", mode='w') as flines:
    json.dump(fp=flines, obj=d3json)
